# RISC-V Baseline Jupyter Notebook

### Import Libraries

In [ ]:
# Import Libraries
from pynq import Overlay
from time import sleep
from pynq.overlays.base import BaseOverlay
from pynq.lib import Pmod_ADC
%matplotlib inline
import matplotlib.pyplot as plt
import struct

### Import Overlay / Set Up Signals

In [ ]:
# Load RISCV2Core Processor
rv = Overlay("RV2Core.bit")
# Define Input Signal (AXI GPIO)
rstN = rv.rstN
step = rv.step
runAll = rv.runAll
hostCtrl = rv.hostCtrl
host_memWr = rv.host_memWr
host_memAdd = rv.host_memAdd
host_datToMem = rv.host_datToMem
# Define Output Signal (AXI GPIO)
datToHost = rv.datToHost
# Define Register Banks
ABI_Array = ['zero','ra','sp','gp','tp','t0','t1','t2','s0/fp','s1','a0','a1','a2','a3','a4','a5','a6','a7',
             's2','s3','s4','s5','s6','s7','s8','s9','s10','s11','t3','t4','t5','t6']

### Project Set Up

In [ ]:
# This block will contain all the neccessary flags and variables

# Print Verbose - Print all information
verbose_prints = True

# Paths to Machine Code
IM0_filename = "asm_examples/swapX3DigitX4WithX5DigitX6.rv"
IM1_filename = "asm_examples/blank.txt"

# Number of Steps to Run
number_of_steps = 300

# Period
period = 2e-08 #20ns

### Read Instruction Memory

In [ ]:
IM0Dat = []
IM1Dat = []

for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2))
    host_memWr.write(0,int('0',2))
    bit = str(RVCore)
    host_memAdd_31_16 = int('0000000000000000',2)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('00',2) #binary
    host_memAdd_12_6 = int('0000000',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    if RVCore == 0:
        print("Instruction Memory of RVCore0\n")
    elif RVCore == 1:
        print("\nInstruction Memory of RVCore1\n")
    for i in range(0,64):
        host_memAdd_5_0 = int('{0:06b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        if RVCore == 0:
            IM0Dat.append(hex(datToHost.read()))
        elif RVCore == 1:
            IM1Dat.append(hex(datToHost.read()))
        if verbose_prints:
            if(hex(datToHost.read()) != "0x0"):
                print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        elif verbose_prints:
            print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        sleep(period) 
    hostCtrl.write(0,int('0',2))
    host_memAdd_15 = int('0',2) 
    host_memAdd_5_0 = int('000000',2) 
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    sleep(10*period)
    
print("\nOperation Finished")

### Read Memory Bank

In [ ]:
MEM0Dat = []
MEM1Dat = []

for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2)) 
    host_memWr.write(0,int('0',2)) 
    bit = str(RVCore)
    host_memAdd_31_16 = int('0000000000000000',2)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('01',2) #binary
    host_memAdd_12_7 = int('000000',2) #binary
    host_memAdd_6_0 = int('0000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:06b}'.format(host_memAdd_12_7) + '{0:07b}'.format(host_memAdd_6_0)
    if RVCore == 0:
        print("Memory of RVCore0\n")
    elif RVCore == 1:
        print("\nMemory of RVCore1\n")
    for i in range(0,68):
        host_memAdd_6_0 = int('{0:07b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:06b}'.format(host_memAdd_12_7) + '{0:07b}'.format(host_memAdd_6_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        if RVCore == 0:
            MEM0Dat.append(hex(datToHost.read()))
        elif RVCore == 1:
            MEM1Dat.append(hex(datToHost.read()))
        if verbose_prints:
            if(hex(datToHost.read()) != "0x0"):
                print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        elif verbose_prints:
            print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        sleep(period)
    hostCtrl.write(0,int('0',2))
    host_memAdd_15 = int('0',2)
    host_memAdd_6_0 = int('0000000',2)
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:06b}'.format(host_memAdd_12_7) + '{0:07b}'.format(host_memAdd_6_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    sleep(10*period)
    
print("\nOperation Finished")

### Read Register Bank

In [ ]:
RB0Dat = []
RB1Dat = []

for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2))
    host_memWr.write(0,int('0',2))
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('10',2) #binary
    host_memAdd_12_5 = int('00000000',2) #binary
    host_memAdd_4_0 = int('00000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:08b}'.format(host_memAdd_12_5) + '{0:05b}'.format(host_memAdd_4_0)
    if RVCore == 0:
        print("Register Bank of RVCore0\n")
    elif RVCore == 1:
        print("\nRegister Bank of RVCore1\n")
    for i in range(0,32):
        host_memAdd_4_0 = int('{0:05b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:08b}'.format(host_memAdd_12_5) + '{0:05b}'.format(host_memAdd_4_0)
        host_memAdd.write(0,int(host_memAdd_str,2)) 
        if RVCore == 0:
            RB0Dat.append(hex(datToHost.read()))
        elif RVCore == 1:
            RB1Dat.append(hex(datToHost.read()))
        regBankStr = "X"+"{:02d}".format(i)
        if verbose_prints:
            if(hex(datToHost.read()) != "0x0"):
                print(regBankStr, '  ', ABI_Array[i], '\tData: ',hex(datToHost.read()))
        elif verbose_prints:
            print(regBankStr, '  ', ABI_Array[i], '\tData: ',hex(datToHost.read()))
        sleep(period)
    hostCtrl.write(0,int('0',2))
    host_memAdd_15 = int('0',2)
    host_memAdd_4_0 = int('00000',2)
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:08b}'.format(host_memAdd_12_5) + '{0:05b}'.format(host_memAdd_4_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    sleep(10*period)
    
print("\nOperation Finished")    

### Read Machine Code from Files

In [ ]:
IM0Array = []; IM1Array = []
for RVCore in range(0,2):
    string = ''; list = []; fullString = ''; data = []; strArray = ''; row = 0
    instrArray = []
    if RVCore == 0:
        file = open(IM0_filename)
        string = file.read(); file.close
        instruction = string.split('\n')
    elif RVCore == 1:
        file = open(IM1_filename)
        string = file.read(); file.close
        instruction = string.split('\n')
    for i in range(0,len(instruction)):
        if (len(instruction[i]) == 10):
            fullString = fullString + instruction[i] + '\n'
        elif(len(instruction[i]) == 8):
            fullString = fullString + "0x" + instruction[i] + '\n'
    for i in range(0,(64-len(instruction))):
            fullString = fullString + '0x00000000' + '\n'
    data = fullString.split('\n')
    for indx in range(0,len(data)-1):
        if indx == 63:
            strArray = strArray + data[indx]
        else:
            strArray = strArray + data[indx] + ", "
        row = row + 1
        if row == 8:
            if indx == 63:
                strArray = strArray
            else:
                strArray = strArray + "\n"
            row = 0     
    if verbose_prints:
        IMArrayStr = 'IM' + str(RVCore) + 'Array = [' + strArray + ']'
        print(IMArrayStr, '\n')
        
    instrArray = strArray.split(", ")
    for i in instrArray:
        if RVCore == 0:
            IM0Array.append(int(i,16))
        elif RVCore == 1:
            IM1Array.append(int(i,16))
print("\nOperation Finished")

### Write Instruction Memory

In [ ]:
for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2))
    host_memWr.write(0,int('1',2))
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('00',2) #binary
    host_memAdd_12_6 = int('0000000',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    if RVCore == 0:
        print("Write to IM0(Instruction Memory of RVCore0)\n")
    elif RVCore == 1:
        print("\nWrite to IM1(Instruction Memory of RVCore1)\n")    
    for i in range(0,64):
        host_memAdd_5_0 = int('{0:06b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        if RVCore == 0:
            host_datToMem.write(0,IM0Array[i])
            if verbose_prints:
                print("Address: ",hex(int(host_memAdd_str,2)), "\t Host Address: ", hex(i), "\tData: ",hex(IM0Array[i]))
        elif RVCore == 1:
            host_datToMem.write(0,IM1Array[i])
            if verbose_prints:
                print("Address: ",hex(int(host_memAdd_str,2)), "\t Host Address: ", hex(i), "\tData: ",hex(IM1Array[i]))
        sleep(period)
    hostCtrl.write(0,int('0',2))
    host_memWr.write(0,int('0',2))
    host_memAdd_15 = int('0',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    sleep(10*period)  
print("\nOperation Finished")

### Write Memory Bank

In [ ]:
# Not fixed yet at all
RVCore = 1
hostCtrl.write(0,int('1',2))
host_memWr.write(0,int('1',2))
bit = str(RVCore)
host_memAdd_15 = int(bit,2) 
host_memAdd_14_13 = int('01',2) 
host_memAdd_12_6 = int('0000000',2) 
host_memAdd_5_0 = int('000000',2) 
host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
host_memAdd.write(0,int(host_memAdd_str,2))
if RVCore == 0:
    print("Memory of RVCore0\n")
elif RVCore == 1:
    print("\nMemory of RVCore1\n")
for i in range(0,32):
    host_memAdd_5_0 = int('{0:06b}'.format(i),2)
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    data = int(1234, 2) # Placeholder for real data
    host_datToMem.write(0,data)
    if verbose_prints:
        print("Host Address: ", hex(i),"\tData: ", hex(data));
    sleep(period)
host_memAdd_15 = int('0',2) #binary
host_memAdd_14_13 = int('00',2) #binary
host_memAdd_5_0 = int('000000',2) #binary
host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
host_memAdd.write(0,int(host_memAdd_str,2))
sleep(10*period)    
print("\nOperation Finished")

### Run Steps

In [ ]:
hostCtrl.write(0,int('0',2))
host_memWr.write(0,int('0',2))
for i in range(0,number_of_steps):
    step.write(0,int('1',2))
    sleep(period)
    step.write(0,int('0',2))
    sleep(period)
print("\nOperation Finished")